In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Getting started

In [ ]:
from kaggle_environments import make, evaluate

# Create the game environment
# Set debug=True to see the errors if your agent refuses to run
env = make("connectx", debug=True)

# List of available default agents
print(list(env.agents))

In [ ]:
# Two random agents play one game round
env.run(["random", "random"])

# Show the game
env.render(mode="ipython")

# Defining agents

Agent should be implemented as a Python function that accepts two arguments: **obs** and **config**. It returns an integer with the selected column, where indexing starts at zero. So, the returned value is one of 0-6, inclusive.

We'll start with a few examples, to provide some context. In the code cell below:

* The first agent behaves identically to the "random" agent above.
* The second agent always selects the middle column, whether it's valid or not! Note that if any agent selects an invalid move, it loses the game.
* The third agent selects the leftmost valid column.

In [ ]:
# Selects random valid column
def agent_random(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return random.choice(valid_moves)

# Selects middle column
def agent_middle(obs, config):
    return config.columns//2

# Selects leftmost valid column
def agent_leftmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return valid_moves[0]

So, what are **obs** and **config**, exactly?

**obs**

obs contains two pieces of information:

* obs.board - the game board (a Python list with one item for each grid location)
* obs.mark - the piece assigned to the agent (either 1 or 2)
obs.board is a Python list that shows the locations of the discs, where the first row appears first, followed by the second row, and so on. We use 1 to track player 1's discs, and 2 to track player 2's discs.


**config**

config contains three pieces of information:

* config.columns - number of columns in the game board (7 for Connect Four)
* config.rows - number of rows in the game board (6 for Connect Four)
* config.inarow - number of pieces a player needs to get in a row in order to win (4 for Connect Four)


# Evaluating agents

In [ ]:
# Agents play one game round
env.run([ agent_leftmost,agent_random])

# Show the game
env.render(mode="ipython")

In [ ]:
# Agents play one game round
env.run([ agent_middle,agent_random])

# Show the game
env.render(mode="ipython")

The outcome of a single game is usually not enough information to figure out how well our agents are likely to perform. To get a better idea, we'll calculate the win percentages for each agent, averaged over multiple games. For fairness, each agent goes first half of the time.

To do this, we'll use the get_win_percentages() function (defined in a hidden code cell). To view the details of this function, click on the "Code" button below.

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

Which agent do you think performs better against the random agent: the agent that always plays in the middle (agent_middle), or the agent that chooses the leftmost valid column (agent_leftmost)?

In [ ]:
get_win_percentages(agent1=agent_middle, agent2=agent_random)

In [ ]:
get_win_percentages(agent1=agent_leftmost, agent2=agent_random)